In [1]:
DEVICES = "GPUS" # Moguce vrijednosti: "CPU", "GPU:0", "GPU:1", "GPUS"
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import logging
# os.environ['TF_CUDNN_DETERMINISTIC']='1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
# 
logging.getLogger('tensorflow').setLevel(logging.FATAL)

# if DEVICES == 'CPU':
    # os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
# elif DEVICES == 'GPU:0':
    # os.environ["CUDA_VISIBLE_DEVICES"] = '0'
# elif DEVICES == 'GPU:1':
    # os.environ["CUDA_VISIBLE_DEVICES"] = '1'


import tensorflow as tf

from dataset.dataset_loader import NerProcessor, FewNERDProcessor, create_tf_dataset_for_client, split_to_tf_datasets, batch_features
from utils.fl_utils import *
from models.model import build_BertNer, MaskedSparseCategoricalCrossentropy
from tokenization import FullTokenizer


from tqdm.notebook import tqdm
import json

import numpy as np

import nest_asyncio
nest_asyncio.apply()

import tensorflow_federated as tff


import bert_modeling


if DEVICES == 'CPU':
    cl_tf_devices = tf.config.list_logical_devices('CPU')
elif DEVICES == 'GPUS':
    cl_tf_devices = tf.config.list_logical_devices('GPU')
# else:
    # cl_tf_devices = tf.config.list_logical_devices('GPU')[:1]
# tff.backends.native.set_local_execution_context(
#     server_tf_device=tf.config.list_logical_devices('CPU')[0],
#     client_tf_devices=cl_tf_devices)


# Pretrained models
TINY = 'uncased_L-2_H-128_A-2'
MODEL_2_256 = 'uncased_L-2_H-256_A-4'
MODEL_2_512 = 'uncased_L-2_H-512_A-8'
MODEL_2_768 = 'uncased_L-2_H-768_A-12'

MODEL_4_128 = 'uncased_L-4_H-128_A-2'
MINI = 'uncased_L-4_H-256_A-4'
SMALL = 'uncased_L-4_H-512_A-8'
MODEL_4_768 = 'uncased_L-4_H-768_A-12'

MODEL_6_128 = 'uncased_L-6_H-128_A-2'
MODEL_6_256 = 'uncased_L-6_H-256_A-4'
MODEL_6_512 = 'uncased_L-6_H-512_A-8'
MODEL_6_768 = 'uncased_L-6_H-768_A-12'

MODEL_8_128 = 'uncased_L-8_H-128_A-2'
MODEL_8_256 = 'uncased_L-8_H-256_A-4'
MEDIUM = 'uncased_L-8_H-512_A-8'
MODEL_8_768 = 'uncased_L-8_H-768_A-12'

MODEL_10_128 = 'uncased_L-10_H-128_A-2'
MODEL_10_256 = 'uncased_L-10_H-256_A-4'
MODEL_10_512 = 'uncased_L-10_H-512_A-8'
MODEL_10_768 = 'uncased_L-10_H-768_A-12'

MODEL_12_128='uncased_L-12_H-128_A-2'
MODEL_12_256 = 'uncased_L-12_H-256_A-4'
MODEL_12_512 = 'uncased_L-12_H-512_A-8'
BASE = 'uncased_L-12_H-768_A-12'


MODEL = os.path.join("models", MODEL_8_256)
SEQ_LEN = 128
BATCH_SIZE = 32
PRETRAINED = True


# processor = NerProcessor('dataset/conll')
processor = FewNERDProcessor('dataset/few_nerd')
datas = processor.data_dir.split('/')[-1]
tokenizer = FullTokenizer(os.path.join(MODEL, "vocab.txt"), True)
train_features = processor.get_train_as_features(SEQ_LEN, tokenizer)
eval_features = processor.get_test_as_features(SEQ_LEN, tokenizer)

# Name of file
modelSize=""
pretr=""

modelSize = {
  'models/uncased_L-2_H-128_A-2': lambda: "tiny_2_128",
  'models/uncased_L-2_H-256_A-4': lambda: "model_2_256",
  'models/uncased_L-2_H-512_A-8': lambda: "model_2_512",
  'models/uncased_L-2_H-768_A-12': lambda: "model_2_768",
  'models/uncased_L-4_H-128_A-2': lambda: "model_4_128",
  'models/uncased_L-4_H-256_A-4': lambda: "mini_4_256",
  'models/uncased_L-4_H-512_A-8': lambda: "small_4_512",
  'models/uncased_L-4_H-768_A-12': lambda: "model_4_768",
  'models/uncased_L-6_H-128_A-2': lambda: "model_6_128",
  'models/uncased_L-6_H-256_A-4': lambda: "model_6_256",
  'models/uncased_L-6_H-512_A-8': lambda: "model_6_512",
  'models/uncased_L-6_H-768_A-12': lambda: "model_6_768",
  'models/uncased_L-8_H-128_A-2': lambda: "model_8_128",
  'models/uncased_L-8_H-256_A-4': lambda: "model_8_256",
  'models/uncased_L-8_H-512_A-8': lambda: "medium_8_512",
  'models/uncased_L-8_H-768_A-12': lambda: "model_8_768",
  'models/uncased_L-10_H-128_A-2': lambda: "model_10_128",
  'models/uncased_L-10_H-256_A-4': lambda: "model_10_256",
  'models/uncased_L-10_H-512_A-8': lambda: "model_10_512",
  'models/uncased_L-10_H-768_A-12': lambda: "model_10_768",
  'models/uncased_L-12_H-128_A-2': lambda: "model_12_128",
  'models/uncased_L-12_H-256_A-4': lambda: "model_12_256",
  'models/uncased_L-12_H-512_A-8': lambda: "model_12_512",
  'models/uncased_L-12_H-768_A-12': lambda: "base_12_768"
}[MODEL]()

if PRETRAINED == True:
    pretr="pretrained"
elif PRETRAINED == False:
    pretr="nontrained"


# Wrap a Keras model for use with TFF.
def model_fn(m_name, num_labels, seq_len, input_spec):
    model = build_BertNer(m_name, num_labels, seq_len)
    for layer in model.layers:
        if (isinstance(layer, bert_modeling.BertModel)):
            layer.trainable = False
    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec,
        loss=MaskedSparseCategoricalCrossentropy()) # reduction=tf.keras.losses.Reduction.NONE))

def eval_model(model, eval_data, do_print=True):
    return evaluate(model, eval_data, 
                    processor.get_label_map(), 
                    processor.token_ind('O'), 
                    processor.token_ind('[SEP]'),
                    processor.token_ind('[PAD]'), 
                    do_print=do_print)
    
# eval_features = eval_features[:5_000] # Samo prvih par tisuca za testiranje, maknuti za konačne ekperimente
eval_data_batched = batch_features(eval_features, processor.get_labels(), SEQ_LEN, tokenizer, batch_size=64)


In [2]:
def state_to_pretrained_model(model_instance, current_state):
    model_instance = restore_model_ckpt(model_instance, MODEL)
    model_instance.layers[-1].set_weights(state.model.trainable)
    return model_instance

In [ ]:
num_clients, num_train_clients = 100, 10
assert num_clients >= num_train_clients

dataset_list = split_to_tf_datasets(train_features, num_clients, batch_size=BATCH_SIZE)
trainer = tff.learning.build_federated_averaging_process(
    model_fn=lambda: model_fn(MODEL, processor.label_len(), SEQ_LEN, dataset_list[0].element_spec),
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(5e-3),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(5e-3),
    use_experimental_simulation_loop=True
)

state = trainer.initialize()
if PRETRAINED:
    pret_model = build_BertNer(MODEL, processor.label_len(), SEQ_LEN)
    for layer in pret_model.layers:
        if (isinstance(layer, bert_modeling.BertModel)):
            layer.trainable = False
    state = state_from_checkpoint(state, pret_model, MODEL)

            
res_list=[]
examples = 0
for rnd_ind in range(1, 501):
    train_data = list(np.random.choice(dataset_list, num_train_clients, replace=False))
    state, metrics = trainer.next(state, train_data)
    print("Round", rnd_ind, "Loss:", metrics['train']['loss'], "Examples:", metrics['stat']['num_examples'])
    examples = metrics['stat']['num_examples']
    # tf.nest.assert_same_structure(state_model, eval_model, check_types=False, expand_composites=True)
    
    # Ne treba svaku rundu gledati tocnost, moze svakih 10 (jedna epoha kumulativno)
    if rnd_ind % num_train_clients == 0:
        state_model = state_to_pretrained_model(build_BertNer(MODEL, processor.label_len(), SEQ_LEN), state)
        # state_model = state_to_model(state, build_BertNer(MODEL, processor.label_len(), SEQ_LEN))
        res = eval_model(state_model, eval_data_batched, do_print=True)
        res['Round'] = rnd_ind
        res['Examples'] = examples
        res_list.append(res)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open("log/results-"+modelSize+"-"+pretr+"-"+datas+"-1.json", "w") as outfile:
    json.dump({'results': res_list, 'model': MODEL, 'seq_len': SEQ_LEN, 
               'pretrained': PRETRAINED, 'batch_size': BATCH_SIZE}, outfile, indent=None, cls=NpEncoder)

Round 1 Loss: 0.7825709 Examples: 13120
Round 2 Loss: 0.6988315 Examples: 13120
Round 3 Loss: 0.6347588 Examples: 13120
Round 4 Loss: 0.5806408 Examples: 13120
Round 5 Loss: 0.53062516 Examples: 13120
Round 6 Loss: 0.50370437 Examples: 13120
Round 7 Loss: 0.47463337 Examples: 13120
Round 8 Loss: 0.44907138 Examples: 13120
Round 9 Loss: 0.4381341 Examples: 13120
Round 10 Loss: 0.42642915 Examples: 13120


Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.1377    0.0571    0.0807      4064
         BUI     0.0326    0.0074    0.0120      5007
         EVE     0.1003    0.0244    0.0392      4104
         LOC     0.4878    0.3590    0.4136     27227
        MISC     0.3819    0.1524    0.2179      9558
         ORG     0.2155    0.2250    0.2202     19077
         PER     0.5525    0.5867    0.5691     21527
         PRO     0.2667    0.1178    0.1634      6230

   micro avg     0.3997    0.3023    0.3442     96794
   macro avg     0.2719    0.1912    0.2145     96794
weighted avg     0.3692    0.3023    0.3240     96794

Round 11 Loss: 0.41570792 Examples: 13120
Round 12 Loss: 0.4065878 Examples: 13120
Round 13 Loss: 0.4007796 Examples: 13120
Round 14 Loss: 0.39098883 Examples: 13120
Round 15 Loss: 0.38885826 Examples: 13120
Round 16 Loss: 0.3858668 Examples: 13120
Round 17 Loss: 0.38012147 Examples: 13120
Round 18 Loss: 0.37820002 Examples: 13120
Round 19 Loss: 0

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.2301    0.2928    0.2577      4064
         BUI     0.1112    0.1348    0.1219      5007
         EVE     0.1729    0.1572    0.1647      4104
         LOC     0.5521    0.5490    0.5505     27227
        MISC     0.3192    0.2541    0.2830      9558
         ORG     0.2826    0.3678    0.3196     19077
         PER     0.6551    0.7896    0.7161     21527
         PRO     0.2894    0.2966    0.2930      6230

   micro avg     0.4283    0.4727    0.4494     96794
   macro avg     0.3266    0.3553    0.3383     96794
weighted avg     0.4296    0.4727    0.4480     96794

Round 21 Loss: 0.36788538 Examples: 13120
Round 22 Loss: 0.36053988 Examples: 13120
Round 23 Loss: 0.3674824 Examples: 13120
Round 24 Loss: 0.3616484 Examples: 13120
Round 25 Loss: 0.35815966 Examples: 13120
Round 26 Loss: 0.36005187 Examples: 13120
Round 27 Loss: 0.35531053 Examples: 13120
Round 28 Loss: 0.35720012 Examples: 13120
Round 29 Loss: 

...

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.3125    0.3984    0.3502      4064
         BUI     0.1504    0.1879    0.1671      5007
         EVE     0.2223    0.2546    0.2374      4104
         LOC     0.5717    0.6878    0.6244     27227
        MISC     0.3362    0.2772    0.3038      9558
         ORG     0.3281    0.4070    0.3633     19077
         PER     0.7007    0.8492    0.7678     21527
         PRO     0.3358    0.3257    0.3307      6230

   micro avg     0.4713    0.5481    0.5068     96794
   macro avg     0.3697    0.4235    0.3931     96794
weighted avg     0.4664    0.5481    0.5027     96794

Round 131 Loss: 0.34424332 Examples: 13120
Round 132 Loss: 0.33755416 Examples: 13120
Round 133 Loss: 0.3384986 Examples: 13120
Round 134 Loss: 0.33645043 Examples: 13120
Round 135 Loss: 0.33912975 Examples: 13120
Round 136 Loss: 0.33894193 Examples: 13120
Round 137 Loss: 0.33578122 Examples: 13120
Round 138 Loss: 0.33932313 Examples: 13120
Round 

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         ART     0.3113    0.3964    0.3487      4064
         BUI     0.1485    0.1734    0.1600      5007
         EVE     0.2164    0.2276    0.2219      4104
         LOC     0.5730    0.6844    0.6237     27227
        MISC     0.3368    0.2813    0.3066      9558
         ORG     0.3253    0.4255    0.3687     19077
         PER     0.7086    0.8449    0.7707     21527
         PRO     0.3354    0.3189    0.3269      6230

   micro avg     0.4718    0.5478    0.5070     96794
   macro avg     0.3694    0.4190    0.3909     96794
weighted avg     0.4676    0.5478    0.5032     96794

Round 141 Loss: 0.3356126 Examples: 13120
Round 142 Loss: 0.33827817 Examples: 13120
Round 143 Loss: 0.33591455 Examples: 13120
Round 144 Loss: 0.33611265 Examples: 13120
Round 145 Loss: 0.3378339 Examples: 13120
Round 146 Loss: 0.33624423 Examples: 13120
Round 147 Loss: 0.3365317 Examples: 13120
Round 148 Loss: 0.33892807 Examples: 13120
Round 14

Evaluating:   0%|          | 0/589 [00:00<?, ?it/s]